# Explanations:

- выборка - тензор из картинок, таргет - вектор силы

---

Авторы используют multi output GPR, настраивая гиперпараметры $\sigma_{cov}$ и $\sigma_{err}$ (можно однозначно их выразить через гиперпараметры из того же sklearn: $l$ и $\sigma$)

GPR - непараметрический метод, суть в том, что мы делаем предположение о виде матрицы корреляции признаков для известных данных.

---

Моделирование в хотя бы немного более сложном случае буду писать на Julia

---

Гиперпараметры:

1)k: Количество элементов в массивах r_cut и p для каждого атома

2)$r_{cut}(i)_j$, i=1..k, j=1..N: векторы r_cut для j атома тоже параметр

3)$p_(i)_j$, i=1..k, j=1..N: векторы p для j атома тоже параметр

4)N_neighbours for summation for IVs

В GPyTorch есть имплементация многоразмерного регрессора: https://docs.gpytorch.ai/en/stable/examples/03_Multitask_Exact_GPs/index.html#multi-output-vector-valued-functions

---

Пока что все размерности предполагаются в системе LJ, потому что пока пытаюсь это зафитить

In [22]:
import random
import os
import time

from numba import jit, njit, vectorize
import numpy as np
import scipy
from numpy.linalg import norm as norm
import pandas as pd

import torch
import torch.nn as nn

In [23]:
def set_seed(seed = 42):
    '''
    
    Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.

    '''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

---

# Hyperparameters:

In [24]:
class CFG:
    '''

    All hyperparameters are here

    '''
    N = 3     # число атомов
    K = 2     # можно называть это разрешением...чем число больше, тем больше размеры матрицы для атомов, фактически это число элементов в наборах p и r_cut

    p = (np.random.rand(K) + 0.1).copy()
    r_cut = (np.random.rand(K) + 0.1).copy()

    N_neig= N - 1 if N != 2 else 1

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

---

Имеется два .csv файла:

1)
| Id(time) | 1_x | 1_y | 1_z | ... | N_z |
|------|-----|-----|-----|-----|-----|
|      |     |     |     |     |     |
|      |     |     |     |     |     |
2)
| Id(time) | f_1_x | f_1_y | f_1_z | ... | f_N_z |
|------|-----|-----|-----|-----|-----|
|      |     |     |     |     |     |
|      |     |     |     |     |     |

Одна строчка отсюда превращается в N матриц (на каждый атом) с N векторами сил

В идеале сделать БДху из двух сущностей: сила и координата, где полями будут их проекции

In [25]:
def create_df_with_coords(coords_file_path = None, forces_file_path = None):
    '''
    just makes df from .csvs with coords and forces
    '''
    coords = pd.read_csv(coords_file_path)

    forces = pd.read_csv(forces_file_path)

    if CFG.N != int(coords.columns[-1][:-1]) + 1:
        raise Exception('Constant N is not equal to amount of particles in .csv')

    return pd.merge(left=coords, right=forces, on='t').drop('t', axis='columns')

df = create_df_with_coords('coords3.csv', 'forces3.csv')
df

,0x,0y,0z,1x,1y,1z,2x,2y,2z,0f_x,0f_y,0f_z,1f_x,1f_y,1f_z,2f_x,2f_y,2f_z
0,2.311403,1.956048,1.756831,1.373676,0.560394,1.412077,0.574069,1.878512,2.598519,0.957439,-0.493213,0.803298,0.185294,0.558420,0.234945,-1.142733,-0.065208,-1.038243
1,2.311389,1.955236,1.756142,1.373495,0.560795,1.413115,0.574264,1.878923,2.598170,0.954511,-0.493660,0.802443,0.186524,0.560881,0.235361,-1.141035,-0.067220,-1.037804
2,2.311375,1.954423,1.755453,1.373314,0.561197,1.414152,0.574459,1.879334,2.597821,0.951574,-0.494118,0.801590,0.187760,0.563350,0.235774,-1.139335,-0.069233,-1.037364
3,2.311361,1.953611,1.754765,1.373132,0.561599,1.415190,0.574654,1.879744,2.597472,0.948629,-0.494584,0.800738,0.189003,0.565828,0.236185,-1.137632,-0.071244,-1.036923
4,2.311348,1.952798,1.754077,1.372951,0.562001,1.416227,0.574849,1.880155,2.597123,0.945674,-0.495060,0.799888,0.190252,0.568314,0.236594,-1.135926,-0.073254,-1.036482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28495,2.586110,2.884062,2.617612,2.806738,1.400084,1.663419,1.750800,0.110808,1.486396,-0.887314,0.551956,-1.581753,-0.437455,-0.877271,0.227353,1.324768,0.325315,1.354400
28496,2.586072,2.883558,2.618508,2.806313,1.400805,1.662541,1.751263,0.110591,1.486378,-0.885575,0.548805,-1.578370,-0.436342,-0.874225,0.225756,1.321916,0.325420,1.352614
28497,2.586033,2.883054,2.619405,2.805887,1.401526,1.661662,1.751726,0.110374,1.486360,-0.883834,0.545680,-1.575000,-0.435234,-0.871203,0.224172,1.319068,0.325524,1.350828
28498,2.585995,2.882550,2.620300,2.805462,1.402247,1.660783,1.752190,0.110157,1.486343,-0.882091,0.542579,-1.571643,-0.434133,-0.868205,0.222602,1.316223,0.325626,1.349041


12 индекс - 1 отн 2

$$
\vec{r_1} = \vec{r_2} + \vec{r}_{12}
$$

$$
\vec{r}_{12} = \vec{r_1} - \vec{r}_{2}
$$

In [26]:
def _get_relative_positions(row, atom_number):
    '''
    This function processes one row of csv into something that we can work with

    Returns np.array matrix that consists of relative positions vectors for passed atom_number to every other atom
    and then we can chose only closest N_neighbours in the next functions
    
    row: df.iloc[row] - typeof(row): pd.Series
    
    returns: Rel_matrix, f_vec
    '''

    s_coord = pd.Series(dtype=float)
    other_atom_numbers = [i for i in range(CFG.N) if i != atom_number]

    for other_numb in other_atom_numbers:
        index = str(atom_number) + str(other_numb)
        for axis in ['x', 'y', 'z']:
            s_coord[index + axis] = row[str(atom_number) + axis] - row[str(other_numb) + axis]

    # we need force vector only for atom_number:
    force_vec = []
    for f_axis in ['f_x', 'f_y', 'f_z']:
        force_vec.append(row[str(atom_number) + f_axis])
        

    Rel_matrix = []
    cur_vector = []

    for (i, elem) in enumerate(s_coord.values):
        if i % 3 == 0 and i != 0:
            Rel_matrix.append(cur_vector)
            cur_vector = []

        cur_vector.append(elem)
    Rel_matrix.append(cur_vector)

    return np.array(Rel_matrix), np.array(force_vec)

In [27]:
_get_relative_positions(df.iloc[0], 1)

(array([[-0.93772712, -1.39565432, -0.34475322],
        [ 0.79960722, -1.31811847, -1.18644141]]),
 array([0.18529392, 0.55842011, 0.23494541]))

In [28]:
from sklearn.preprocessing import normalize

In [29]:
@njit(fastmath=True)
def make_one_vec_transformed(vec, vec_norm, r_cut_i, p_i):
    '''
    vec: np.array - normalized vector
    norm: its norm
    r_cut_i: i-th component of
    '''
    return vec * np.exp(
        -np.power((vec_norm / r_cut_i), p_i)
        )

make_matrix_transformed = np.vectorize(make_one_vec_transformed)

def create_V_i(i, normalized_m, norms, r_cut=CFG.r_cut, p=CFG.p):
    '''
    normalized_m: matrix of relative distances, where rows - normalized vectors
    i: i-th component of r_cut and p, i in range 1..K (or in 0..K-1 in code)
    '''
    transf_vecs = make_matrix_transformed(normalized_m, norms[:, np.newaxis], r_cut[i], p[i])

    return np.sum(transf_vecs, axis=0)

# @njit(parallel=True)
def create_V(normalized_m, norms, K=CFG.K):
    '''
    creates V
    '''
    V = []
    for i in range(K):
        V.append(
            create_V_i(i, normalized_m, norms)
        )

    return np.array(V)

In [30]:
# @njit(
#     # parallel=True,
#     # fastmath=True
#     )
def _calculate_matrix_for_atom(relative_distances, r_cut=CFG.r_cut, p=CFG.p, N_neig=CFG.N_neig, K=CFG.K):
    '''

    relative_distances: np.array matrix of relative distance vectors

    '''

    norms = norm(relative_distances, axis=-1)
    
    # Only closest N_neig are counting:
    indexlist = np.argsort(norm(relative_distances, axis=1))
    relative_distances = relative_distances[indexlist[len(relative_distances) - N_neig:]]

    normalized_rel_distances = relative_distances / norms[:, np.newaxis]

    # print(
    #     create_V_i(0, normalized_rel_distances, norms), f'{CFG.r_cut=}, {CFG.p=}'
    # )

    V = create_V(normalized_rel_distances, norms)
    
    A = V / norm(V, axis=-1)[:, np.newaxis]

    X = V @ A.T

    return X

In [31]:
def get_matrix_for_atom(row = None, atom_number = None):
    '''

    This function will create X matrix for passed atom with
    arrays of r_cut and p of length k

    It is a wrapper for _get_relative_positions and _calculate_matrix_for_atom, so I can speed up matrix calculations
    with numba for _calculate_matrix_for_atom

    atom_number: a number of atom that we are passing
    row: one row from df_with_coords, i.e. df.iloc[index_of_row]

    '''

    # creating row of relative coordinates for concrete atom:
    relative_distances, f_vec = _get_relative_positions(row=row, atom_number=atom_number)
    X = _calculate_matrix_for_atom(relative_distances=relative_distances)
    
    return X, f_vec

# %timeit get_matrix_for_atom(row=df.iloc[0], atom_number=1)

get_matrix_for_atom(row=df.iloc[0], atom_number=1)

(array([[0.39852586, 0.39714546],
        [0.11066237, 0.11104701]]),
 array([0.18529392, 0.55842011, 0.23494541]))

In [32]:
from tqdm import tqdm
import gc

**У нас будет train и val выборки, все-таки выборку, для который известен таргет принято называть validation, на которой мы качество оцениваем, а test это все-таки выборка, для который неизвестны таргеты**

In [33]:
def create_tensor_dataset(coords_file_path = 'coords.csv', forces_file_path = 'forces.csv', step=1, transform=None):
    '''

    Примитивная версия датасета, просто все будет хранить в одном тензоре...

    Эта функция - wrapper на все выше написанные функции, она по переданным путям к .csv
    возвращает тензор из матриц для каждого атома в каждой строчке и тензор из векторов сил


    ИНогда есть смысл делать побольше шаг между соседними строчками, поскольку если есть почти одинаковые матрицы, то
    это по-сути линейная зависимость и модель тогда надо сильнее регулизировать

    transform:

    '''

    X_data = []
    Y = []

    df = create_df_with_coords(coords_file_path=coords_file_path, forces_file_path=forces_file_path)#.loc[range(1000), :]
    row_indexes = [i for i in range(0, len(df.index), step)]

    for atom_number in range(CFG.N):
        for index in tqdm(row_indexes, desc=f'Progress for atom {atom_number}'):
            row = df.iloc[index]
            x, f = get_matrix_for_atom(row=row, atom_number=atom_number)
            X_data.append(x)
            Y.append(f)

    gc.collect()

    #TODO:
    if transform:
        pass

    #TODO:
    # Все-таки стоит возвращать датасет в виде тензор из:(x_sample, y_sample)

    return torch.tensor(X_data, dtype=torch.double), torch.tensor(Y, dtype=torch.double)

In [34]:
X, Y = create_tensor_dataset('coords3.csv', 'forces3.csv', step=10)

Progress for atom 2: 100%|██████████| 2850/2850 [00:06<00:00, 428.30it/s]


Пока никакие параметры особо не надо настраивать, поэтому и кросс валидацию не буду делать пока что, затем ее можно сделать, передавая в функцию create_dataloaders еще один параметр - фолд, на котором трейн, предварительно поделив на фолды датасет

Если просто брать в качестве трейна другие строчки из одной генерации, то можно не отследить переобучения, стоит пробовать тестить на датасете, который отдельно сгенерирован с таким же числом частиц, который модель еще вообще не видела

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

## Когда молекул уже будет много как хранить данные:

In [36]:
# Эта клетка нужна будет, когда молекул будет много (N > 100, K порядка 100)

def create_df_with_paths(df_coords: pd.DataFrame, first_folder = 'Atom_matrices'):
    '''

    Пока эта функция не нужна, но в будущем за счет нее как раз будет работать PathBasedDataset

    gets df, returns df with paths to torch matrices for each atom for different times,
    basically this function will call get_matrix_for_atom a lot of times

    output: pd.DataFrame that orignated from this:
    
    | Index | 1_atom_X_path                     | ... | N_atom_X_path                     |
    |-------|-----------------------------------|-----|-----------------------------------|
    | 1     | ./atom_matrices/index1/atom1.tb   |     | ./atom_matrices/index1/atomN.tb   |
    | ...   |                                   |     |                                   |
    | 30k   | ./atom_matrices/index30k/atom1.tb |     | ./atom_matrices/index30k/atomN.tb |
    
    but eventually will look like this:

    | Index   | atom_X_path                       |
    |---------|-----------------------------------|
    | 1       | ./atom_matrices/index1/atom1.tb   |
    | ...     | ...                               |
    | 30k * N | ./atom_matrices/index30k/atomN.tb |

    '''
    row_numbers = df_coords.index

    df_paths = pd.DataFrame(
        {
            'path': []
        }
    )

    pass

class PathBasedDataset(torch.utils.data.Dataset):
    '''

    Это будет класс датасета из торча для большого числа молекул, если молекул будет очень много, то надо будет уже хранить все матрицы X не в оперативной памяти

    При создании экземпляра будет передаваться pd.Dataframe, который
    состоит из трех колонок - проекций вектора силы и еще одной колонки - путь к файлу, где лежит как-то заэнкоженная
    матрица для данного атома, и так для каждого атома (я проверил, что запись и чтение при помощи torch.save и torch.load для тензоров очень быстрое)

    '''
    def __init__(self, df, transforms=None, mode='train'):
        self.df = df    # it will be dataframe with coordinates and forces of all atoms
        self.mode = mode
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        x = 1   # it will be a matrix KxK for each atom
        y = 1   # it will be a force vector with shape: (3)

        if self.mode == 'test':
            return x
        return x, y

In [39]:
from torch.utils.data import DataLoader

In [43]:
def create_dataloaders(train_dataset, val_dataset, train_bs=64, val_bs=128, fold=None):
    '''

    Returns train_loader, val_loader

    fold: will be used in cross validation, when I will implement it

    '''
    
    train_loader = DataLoader(dataset=train_dataset, batch_size=train_bs)

    val_loader = DataLoader(dataset=val_dataset, batch_size=val_bs)

    return train_loader, val_loader

---

# Обучение: